In [1]:
import pandas as pd
import numpy as np
import os
import yt_dlp
import whisper

pd.set_option('display.max_rows', 20)

In [2]:
# Only if you have 
import torch
print("CUDA Availability: ")
print(torch.cuda.is_available())  # Returns True if CUDA is available: GPU is available && CUDA is installed and configured
print(torch.cuda.get_device_name(0))  # Returns GPU name if available

CUDA Availability: 
True
NVIDIA GeForce RTX 2060


In [5]:
# Define directories for audio and transcripts
AUDIO_DIR = "audio_files"
TRANSCRIPT_DIR = "transcripts"
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(TRANSCRIPT_DIR, exist_ok=True)

In [12]:
def is_livestream_or_long(video_id, max_duration=3600): # 1 hour
    """Checks if a video is a livestream using yt-dlp metadata."""
    url = f"https://www.youtube.com/watch?v={video_id}"
    
    ydl_opts = {
        'quiet': True,
        'simulate': True,
        'force_generic_extractor': False,
        'no_warnings': True,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)  # Get metadata without downloading

        # Check if the video is a livestream
        if info.get("is_live", False):
            print(f"⚠️ Skipping {video_id}: It is a livestream.")
            return True  # Indicate the video should be skipped
        
        # Check if the video duration is too long
        duration = info.get("duration", 0)  # Duration in seconds
        if duration and duration > max_duration:
            print(f"⚠️ Skipping {video_id}: It is too long ({duration // 3600}h {duration % 3600 // 60}m).")
            return True  # Indicate the video should be skipped

        return False  # Video is valid
    except Exception as e:
        print(f"⚠️ Video {video_id} is unavailable or removed. Skipping.")
        return None  # Returning None to indicate failure

def download_audio(video_id, output_format="mp3", failed_videos=None):
    """Downloads audio from a YouTube video and saves it in the audio directory."""
    
    if failed_videos is None:
        failed_videos = []  # Prevents NoneType issues

    # **Check if the video is a livestream**
    if is_livestream_or_long(video_id):
        print(f"⚠️ Skipping {video_id}: Not suitable for transcription.")
        failed_videos.append(video_id)  # Add to failed list
        return None  # Prevent downloading
    
    try:
        url = f"https://www.youtube.com/watch?v={video_id}"
        output_filename = os.path.join(AUDIO_DIR, f"{video_id}.%(ext)s")  # Save in AUDIO_DIR

        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': output_filename,
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': output_format,
                'preferredquality': '192',
            }],
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        # Check for incorrect double extension
        expected_filename = os.path.join(AUDIO_DIR, f"{video_id}.{output_format}")
        double_extension = os.path.join(AUDIO_DIR, f"{video_id}.{output_format}.{output_format}")

        if os.path.exists(double_extension):
            os.rename(double_extension, expected_filename)

        return expected_filename
    except Exception as e:
        print(f"Error downloading audio for video {video_id}: {e}")
        return None


def transcribe_audio(audio_file):
    device = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available
    model = whisper.load_model("base", device=device)  # Load Whisper on GPU with base model
    result = model.transcribe(audio_file)
    return result["text"]


def save_transcript(video_id, transcript_text):
    output_filename = os.path.join(TRANSCRIPT_DIR, f"{video_id}.txt")
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(transcript_text)
    print(f"Transcript saved at: {output_filename}")

In [25]:
## Transcription Cell

df = pd.read_csv("transcriptless_videos_11.csv") # Replace with your CSV file

failed_videoes = []

for index, row in df.iterrows():
    video_id = row["Video Id"]  # Extract Video ID
    print(f"\nProcessing Video ID: {video_id}")

    audio_file = download_audio(video_id, failed_videos=failed_videoes)

    if audio_file is None:
        print(f"Skipping {video_id}: Download failed.")
        failed_videoes.append(video_id)
        print("Total failed videos count:", len(failed_videoes))
        continue  # Skip to the next video

    if os.path.exists(audio_file):
        print(f"File exists: {audio_file}")
        try:
            transcript = transcribe_audio(audio_file)
            save_transcript(video_id, transcript)
        except Exception as e:
            print(f"Error during transcription for {video_id}: {e}")
            failed_videoes.append(video_id)
            print("Total failed videos count:", len(failed_videoes))
    else:
        print(f"Error: Audio file {audio_file} not found! Skipping.")
        failed_videoes.append(video_id)
        print("Total failed videos count:", len(failed_videoes))


if failed_videoes:
    print("\n⚠️ The following videos failed to generate transcripts:")
    for vid in failed_videoes:
        print(f"- {vid}")
else:
    print("\n✅ All videos were successfully processed!")


Processing Video ID: 9fSVEGI4i04


ERROR: [youtube] 9fSVEGI4i04: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


⚠️ Video 9fSVEGI4i04 is unavailable or removed. Skipping.
[youtube] Extracting URL: https://www.youtube.com/watch?v=9fSVEGI4i04
[youtube] 9fSVEGI4i04: Downloading webpage
[youtube] 9fSVEGI4i04: Downloading tv client config
[youtube] 9fSVEGI4i04: Downloading player e7567ecf
[youtube] 9fSVEGI4i04: Downloading tv player API JSON
[youtube] 9fSVEGI4i04: Downloading ios player API JSON


ERROR: [youtube] 9fSVEGI4i04: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Error downloading audio for video 9fSVEGI4i04: ERROR: [youtube] 9fSVEGI4i04: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
Skipping 9fSVEGI4i04: Download failed.
Total failed videos count: 1

Processing Video ID: ZuUJBRDUS30
[youtube] Extracting URL: https://www.youtube.com/watch?v=ZuUJBRDUS30
[youtube] ZuUJBRDUS30: Downloading webpage
[youtube] ZuUJBRDUS30: Downloading tv client config
[youtube] ZuUJBRDUS30: Downloading player e7567ecf
[youtube] ZuUJBRDUS30: Downloading tv player API JSON
[youtube] ZuUJBRDUS30: Downloading ios player API JSON
[youtube] ZuUJBRDUS30: Downloading m3u8 information
[info] ZuUJBRDUS30: Downloading 1 format(s): 251
[download] De

...

Error downloading audio for video ohtYxA21Yk0: ERROR: [youtube] ohtYxA21Yk0: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
Skipping ohtYxA21Yk0: Download failed.
Total failed videos count: 7

Processing Video ID: AbXuVrTVK-s
[youtube] Extracting URL: https://www.youtube.com/watch?v=AbXuVrTVK-s
[youtube] AbXuVrTVK-s: Downloading webpage
[youtube] AbXuVrTVK-s: Downloading tv client config
[youtube] AbXuVrTVK-s: Downloading player e7567ecf
[youtube] AbXuVrTVK-s: Downloading tv player API JSON
[youtube] AbXuVrTVK-s: Downloading ios player API JSON
[youtube] AbXuVrTVK-s: Downloading m3u8 information
[info] AbXuVrTVK-s: Downloading 1 format(s): 251
[download] De

ERROR: [youtube] tTT5kCgce0w: Video unavailable


⚠️ Video tTT5kCgce0w is unavailable or removed. Skipping.
[youtube] Extracting URL: https://www.youtube.com/watch?v=tTT5kCgce0w
[youtube] tTT5kCgce0w: Downloading webpage
[youtube] tTT5kCgce0w: Downloading tv client config
[youtube] tTT5kCgce0w: Downloading player e7567ecf
[youtube] tTT5kCgce0w: Downloading tv player API JSON
[youtube] tTT5kCgce0w: Downloading ios player API JSON


ERROR: [youtube] tTT5kCgce0w: Video unavailable


Error downloading audio for video tTT5kCgce0w: ERROR: [youtube] tTT5kCgce0w: Video unavailable
Skipping tTT5kCgce0w: Download failed.
Total failed videos count: 8

Processing Video ID: kDEUVDMP-C4
[youtube] Extracting URL: https://www.youtube.com/watch?v=kDEUVDMP-C4
[youtube] kDEUVDMP-C4: Downloading webpage
[youtube] kDEUVDMP-C4: Downloading tv client config
[youtube] kDEUVDMP-C4: Downloading player e7567ecf
[youtube] kDEUVDMP-C4: Downloading tv player API JSON
[youtube] kDEUVDMP-C4: Downloading ios player API JSON
[youtube] kDEUVDMP-C4: Downloading m3u8 information
[info] kDEUVDMP-C4: Downloading 1 format(s): 251
[download] Destination: audio_files\kDEUVDMP-C4.webm
[download] 100% of    1.93MiB in 00:00:01 at 1.34MiB/s   
[ExtractAudio] Destination: audio_files\kDEUVDMP-C4.mp3
Deleting original file audio_files\kDEUVDMP-C4.webm (pass -k to keep)
File exists: audio_files\kDEUVDMP-C4.mp3
Transcript saved at: transcripts\kDEUVDMP-C4.txt

Processing Video ID: y2dS39mU7JI
[youtube] Extrac

In [21]:
# Resume Transcription Cell

# 🔹 Replace with the actual video ID where the script stopped
last_processed_video_id = "IoYPcfXDOW4"  # Replace with the correct ID

# 🔹 Find the index where this video ID is located
start_index = df[df["Video Id"] == last_processed_video_id].index[0] + 1  # Move to the next row

# 🔹 Resume from the next row
for index, row in df.iloc[start_index:].iterrows():  # Start from the saved index
    video_id = row["Video Id"]
    print(f"\nProcessing Video ID: {video_id}")

    audio_file = download_audio(video_id, failed_videos=failed_videoes)

    if audio_file is None:
        print(f"Skipping {video_id}: Download failed.")
        failed_videoes.append(video_id)
        print("Total failed videos count:", len(failed_videoes))
        continue  # Skip to the next video

    if os.path.exists(audio_file):
        print(f"File exists: {audio_file}")
        try:
            transcript = transcribe_audio(audio_file)
            save_transcript(video_id, transcript)
        except Exception as e:
            print(f"Error during transcription for {video_id}: {e}")
            failed_videoes.append(video_id)
            print("Total failed videos count:", len(failed_videoes))
    else:
        print(f"Error: Audio file {audio_file} not found! Skipping.")
        failed_videoes.append(video_id)
        print("Total failed videos count:", len(failed_videoes))


if failed_videoes:
    print("\n⚠️ The following videos failed to generate transcripts:")
    for vid in failed_videoes:
        print(f"- {vid}")
else:
    print("\n✅ All videos were successfully processed!")


Processing Video ID: q4v9THvRsEc


ERROR: [youtube] q4v9THvRsEc: Video unavailable


⚠️ Video q4v9THvRsEc is unavailable or removed. Skipping.
[youtube] Extracting URL: https://www.youtube.com/watch?v=q4v9THvRsEc
[youtube] q4v9THvRsEc: Downloading webpage
[youtube] q4v9THvRsEc: Downloading tv client config
[youtube] q4v9THvRsEc: Downloading player e7567ecf
[youtube] q4v9THvRsEc: Downloading tv player API JSON
[youtube] q4v9THvRsEc: Downloading ios player API JSON


ERROR: [youtube] q4v9THvRsEc: Video unavailable


Error downloading audio for video q4v9THvRsEc: ERROR: [youtube] q4v9THvRsEc: Video unavailable
Skipping q4v9THvRsEc: Download failed.
Total failed videos count: 138

Processing Video ID: cvESrYGooks


...

Error downloading audio for video 9TPt8yR2RxI: ERROR: [youtube] 9TPt8yR2RxI: Video unavailable
Skipping 9TPt8yR2RxI: Download failed.
Total failed videos count: 148

Processing Video ID: hO8DBCJyIQc
[youtube] Extracting URL: https://www.youtube.com/watch?v=hO8DBCJyIQc
[youtube] hO8DBCJyIQc: Downloading webpage
[youtube] hO8DBCJyIQc: Downloading tv client config
[youtube] hO8DBCJyIQc: Downloading player e7567ecf
[youtube] hO8DBCJyIQc: Downloading tv player API JSON
[youtube] hO8DBCJyIQc: Downloading ios player API JSON
[youtube] hO8DBCJyIQc: Downloading m3u8 information
[info] hO8DBCJyIQc: Downloading 1 format(s): 251
[download] Destination: audio_files\hO8DBCJyIQc.webm
[download] 100% of  815.78KiB in 00:00:00 at 4.50MiB/s   
[ExtractAudio] Destination: audio_files\hO8DBCJyIQc.mp3
Deleting original file audio_files\hO8DBCJyIQc.webm (pass -k to keep)
File exists: audio_files\hO8DBCJyIQc.mp3
Transcript saved at: transcripts\hO8DBCJyIQc.txt

Processing Video ID: 6j-CuKEQGEg
[youtube] Extr

ERROR: [youtube] _Z_0ov7PX2k: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


⚠️ Video _Z_0ov7PX2k is unavailable or removed. Skipping.
[youtube] Extracting URL: https://www.youtube.com/watch?v=_Z_0ov7PX2k
[youtube] _Z_0ov7PX2k: Downloading webpage
[youtube] _Z_0ov7PX2k: Downloading tv client config
[youtube] _Z_0ov7PX2k: Downloading player e7567ecf
[youtube] _Z_0ov7PX2k: Downloading tv player API JSON
[youtube] _Z_0ov7PX2k: Downloading ios player API JSON


ERROR: [youtube] _Z_0ov7PX2k: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Error downloading audio for video _Z_0ov7PX2k: ERROR: [youtube] _Z_0ov7PX2k: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
Skipping _Z_0ov7PX2k: Download failed.
Total failed videos count: 149

Processing Video ID: RpGFSiyTCMA
[youtube] Extracting URL: https://www.youtube.com/watch?v=RpGFSiyTCMA
[youtube] RpGFSiyTCMA: Downloading webpage
[youtube] RpGFSiyTCMA: Downloading tv client config
[youtube] RpGFSiyTCMA: Downloading player e7567ecf
[youtube] RpGFSiyTCMA: Downloading tv player API JSON
[youtube] RpGFSiyTCMA: Downloading ios player API JSON
[youtube] RpGFSiyTCMA: Downloading m3u8 information
[info] RpGFSiyTCMA: Downloading 1 format(s): 251
[download] 